In [1]:


import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
import joblib

cols = ["target", "id", "date_time", "query", "user_id", "comment"]

comments = pd.read_csv(r"C:\Users\Madhukar Nirgun\Downloads\archive\tweets.csv",encoding="latin1",names=cols)
comments

# comments.columns


comments.head(10)

comments["target"].value_counts()

comments['target'] = comments['target'].replace(4, 1)
comments['target'].value_counts()

import re

def clean_comments(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)  # cooool → cool
    return text.strip()

comments['clean_comments'] = comments['comment'].apply(clean_comments)


comments

X = comments['clean_comments']
y = comments['target']

tfidf = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1, 2),
    stop_words='english',
    sublinear_tf=True
)

X_tfidf = tfidf.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_tfidf,y, test_size=0.2,random_state=14,stratify=y)

svm_model = LinearSVC(C=2,class_weight='balanced',random_state=14)

svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


joblib.dump(svm_model, "models/svm_model.pkl")
joblib.dump(tfidf, "models/vectorizer.pkl")


tfidf = TfidfVectorizer(
    max_features=15000,        # VERY IMPORTANT
    ngram_range=(1, 2),
    stop_words='english',
    min_df=5,
    max_df=0.9
)

X_tfidf = tfidf.fit_transform(X)



Accuracy: 0.78044375
              precision    recall  f1-score   support

           0       0.80      0.75      0.77    160000
           1       0.77      0.81      0.79    160000

    accuracy                           0.78    320000
   macro avg       0.78      0.78      0.78    320000
weighted avg       0.78      0.78      0.78    320000



In [2]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    objective='binary:logistic',
    n_estimators=300,
    max_depth=6, learning_rate=0.1,subsample=0.8,colsample_bytree=0.8,tree_method='hist',eval_metric='logloss',n_jobs=-1,random_state=14)

xgb_model.fit(X_train, y_train)


y_pred = xgb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

joblib.dump(xgb_model, "models/xgb_model.pkl")

Accuracy: 0.731090625
              precision    recall  f1-score   support

           0       0.79      0.62      0.70    160000
           1       0.69      0.84      0.76    160000

    accuracy                           0.73    320000
   macro avg       0.74      0.73      0.73    320000
weighted avg       0.74      0.73      0.73    320000



['models/xgb_model.pkl']

In [ ]:
dk = 